## 

# What happens when not using an MLflow format model with MLflow

In [ ]:
model_input = {"inputs": 
[[0.0126481372762872, 0.0506801187398187, 0.00241654245523897, 0.0563010619323185, 0.0273260502020124, 
0.0171618818193638, 0.0412768238419757, -0.0394933828740919, 0.00371173823343597, 0.0734802269665584], 
[-0.107225631607358, -0.044641636506989, -0.0773415510119477, -0.0263278347173518, -0.0896299427450836, 
-0.0961978613484469, 0.0265502726256275, -0.076394503750001, -0.0425721049227942, -0.0052198044153011], 
[0.0271782910803654, 0.0506801187398187, -0.0353068801305926, 0.0322009670761646, -0.0112006298276192, 
0.00150445872988718, -0.0102661054152432, -0.00259226199818282, -0.0149564750249113, -0.0507829804784829], 
[-0.00551455497881059, 0.0506801187398187, 0.00133873038135806, -0.0848566365108683, -0.0112006298276192, 
-0.0166581520539057, 0.0486400994501499, -0.0394933828740919, -0.0411803851880079, -0.0880619427119953], 
[0.0671362140415805, 0.0506801187398187, 0.0207393477112143, -0.00567061055493425, 0.0204462859110067, 
0.0262431872112602, -0.0029028298070691, -0.00259226199818282, 0.00864028293306308, 0.00306440941436832]]
}

import mlflow
model = mlflow.pyfunc.load_model("model.pkl")
model.predict(model_input)

### OOoo, this is not working, it is not an MLFlow model... Let me make it one.

# Utilizing MLflow and AML for any model

AzureML now support only 3 types of models [Custom, MLflow and Trition]

In the situtaion you want to utilize the benefits of an MLflow model such as:
-  No code deployment
- ??
- ??

However, you didn't train your model with the MLflow APIs or you are using a ML framework not support by MLFlow, you you will need to convert your custom model to MLFLow conform to the MLFlow format.

# Changing a model to MLFLow flow

MLflow provides convenient functions for creating models with the pyfunc flavor in a variety of machine learning frameworks (scikit-learn, Keras, Pytorch, and more); however, they do not cover every use case. For example, you may want to create an MLflow model with the pyfunc flavor using a framework that MLflow does not natively support or you may want to change a pre-trained model trainined out of MLFlow into an MLFlow formated model

MLflow recommeneds the following [process](https://mlflow.org/docs/latest/python_api/mlflow.pyfunc.html#creating-custom-pyfunc-models) to change your model:

1) Create a Python Wrapper for your model.

2) Specify the Conda environment for your model.

3) Use the save_model() and log_model() methods for creating custom pyfunc models that incorporate custom inference logic and artifacts that the logic may require.

## Create a Python Wrapper

In [26]:
# Load training and test datasets
from sys import version_info
import sklearn
from sklearn import datasets
from sklearn.model_selection import train_test_split

PYTHON_VERSION = "{major}.{minor}.{micro}".format(major=version_info.major,
                                                  minor=version_info.minor,
                                                  micro=version_info.micro)

# Train and save an SKLearn model
sklearn_model_path = "model.pkl"

# Create an `artifacts` dictionary that assigns a unique name to the saved XGBoost model file.
# This dictionary will be passed to `mlflow.pyfunc.save_model`, which will copy the model file
# into the new MLflow Model's directory.
artifacts = {
    "sklearn_model": sklearn_model_path
}

import mlflow.pyfunc
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

class SKLearnWrapper(mlflow.pyfunc.PythonModel):

    def load_context(self, context):
        import pickle
        self.sklearn_model = pickle.load(open(context.artifacts["sklearn_model"], 'rb'))
    
    def predict(self, model, data):
        return self.sklearn_model.predict(data)

# Create a Conda environment for the new MLflow Model that contains all necessary dependencies.

In [ ]:
import cloudpickle
conda_env = {
    'channels': ['defaults'],
    'dependencies': [
      'python={}'.format(PYTHON_VERSION),
      'pip',
      {
        'pip': [
          'mlflow',
          'scikit-learn=={}'.format(sklearn.__version__),
          'cloudpickle=={}'.format(cloudpickle.__version__),
        ],
      },
    ],
    'name': 'sklearn_env'
}

# Save the MLflow Model to disk

In [ ]:
mlflow_pyfunc_model_path = "sklearn_mlflow_pyfunc7"
mlflow.pyfunc.save_model(path=mlflow_pyfunc_model_path, python_model=SKLearnWrapper(), conda_env=conda_env, artifacts=artifacts)

# Load the MLFlow formated model and test predictions

In [ ]:
loaded_model = mlflow.pyfunc.load_model(mlflow_pyfunc_model_path)

# Evaluate the model
import pandas as pd
test_predictions = loaded_model.predict(model_input)
print(test_predictions)

## Compare to your non MLFlow formated model
It should be the same prediction

In [28]:
# load the model from disk
import pickle
loaded_model = pickle.load(open(sklearn_model_path, 'rb'))
result = loaded_model.predict(model_input)
print(result)

[4634.10508517]


# Register the model
Start dummy run, model to run and save to registry

In [ ]:
# mlflow.pyfunc.log_model(artifact_path=mlflow_pyfunc_model_path, loader_module=None, data_path=None, code_path=None,
#                              python_model=SKLearnWrapper(),
#                              registered_model_name="Custom_mlflow_model", conda_env=conda_env, artifacts=artifacts)
# mlflow.end_run()